This script is for loading and saving the image with a scale bar.

# Import libraries

You might need to install the scalebar library: $\mathtt{pip\; install\; matplotlib-scalebar}$

In [ ]:
# Basic
import os, sys
from os.path import join
from importlib import reload

# Data
import numpy as np
import pandas as pd
import h5py

# Plotting
import imageio
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

# skimage
import skimage as sk
from skimage.registration import phase_cross_correlation

# scipy
from scipy.ndimage import gaussian_filter, fourier_shift

# self-written library
sys.path.append(join(os.getcwd(), "library"))
import pymaxymus as mx

In [ ]:
# interactive plotting (WIDGET for jupyter lab, NOTEBOOK for jupyter notebook)
import ipywidgets

%matplotlib widget

# Auto formatting
%load_ext jupyter_black

# Experiment Specific functions

In [ ]:
def FFT(image):
    """
    Fourier transform
    -------
    author: CK 2021
    """
    return np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(image)))

In [ ]:
# scipy
from scipy.ndimage import fourier_shift
from scipy.ndimage import shift as scipy_shift


def shift_image(image, shift, interpolation=True, out_dtype="numpy"):
    """
    Shifts image with sub-pixel precission in Fourier space


    Parameters
    ----------
    image: cupy array
        Moving image, will be shifted by shift vector

    shift: vector
        x and y translation in px

    out_dtype : string
        output data as numpy or cupy array

    Returns
    -------
    image_shifted: cupy/numpy array
        Shifted image
    -------
    author: CK 2023
    """

    # Shift Image
    image = np.array(image)

    if interpolation is True:
        shift_image = scipy_shift(image, shift, mode="reflect")
    else:
        shift_image = fourier_shift(np.fft.fft2(image), shift)
        shift_image = np.fft.ifft2(shift_image)
        shift_image = shift_image.real

    return shift_image


def image_registration(image_unproccessed, image_background, roi=None, im_out=False):
    """
    Aligns two images with sub-pixel precission through image registration


    Parameters
    ----------
    image_unproccessed: cupy array
        Moving image, will be aligned with respect to image_background

    image_background: cupy array
        static reference image

    roi: region of interest defining the region of the images used to calc
        the alignment

    im_out: bool
        return also corrected image if true

    Returns
    -------
    image_corrected: array
        Shifted/aligned moving image
    shift: array
        shift (dy,dx)
    -------
    author: CK 2022
    """

    # Calculate Shift
    if roi == None:
        shift, error, diffphase = phase_cross_correlation(
            image_background, image_unproccessed, upsample_factor=100
        )
    else:
        shift, error, diffphase = phase_cross_correlation(
            image_background[
                roi[2] : roi[3],
                image_unproccessed[roi[2] : roi[3], roi[0] : roi[1]],
                roi[0] : roi[1],
            ],
            upsample_factor=100,
        )

    # print(f'Detected pixel shift (y, x): {shift}')

    if im_out == True:
        # Shift Image
        image_corrected = shift_image(image_unproccessed, shift)

        return image_corrected, shift
    else:
        return shift

In [ ]:
from ipywidgets import FloatRangeSlider, FloatSlider, Button, interact, IntSlider


def cimshow(im, **kwargs):
    """Simple 2d image plot with adjustable contrast.

    Returns matplotlib figure and axis created.
    """
    im = np.array(im)
    fig, ax = plt.subplots()
    im0 = im[0] if len(im.shape) == 3 else im
    mm = ax.imshow(im0, **kwargs)

    cmin, cmax, vmin, vmax = np.nanpercentile(im, [0.1, 99.9, 0.001, 99.999])
    # vmin, vmax = np.nanmin(im), np.nanmax(im)
    sl_contrast = FloatRangeSlider(
        value=(cmin, cmax),
        min=vmin,
        max=vmax,
        step=(vmax - vmin) / 500,
        layout=ipywidgets.Layout(width="500px"),
    )

    @ipywidgets.interact(contrast=sl_contrast)
    def update(contrast):
        mm.set_clim(contrast)

    if len(im.shape) == 3:
        w_image = IntSlider(value=0, min=0, max=im.shape[0] - 1)

        @ipywidgets.interact(nr=w_image)
        def set_image(nr):
            mm.set_data(im[nr])

    return fig, ax

In [ ]:
def max_plot(
    data,
    destination,
    pixel_size,
    length_fraction,
    color,
    location=1,
    units="nm",
    cmap="gray",
    size=(2, 2),
    save=True,
    scale=(0, 100),
    rang=(0, -1),
):
    """
    Plot single image recorded at the MAXYMUS microscope at BESSY.
    INPUT:
        data: data set
        destination: filename where to save the image
        pixel_size: size of one pixel in nm (otherwise change units)
        length_fraction: length of the scale bar
        color: color of the scale bar ('w' for white and 'k' for black)
        location: location of the scale bar (default is 1, right upper corner)
        units: unit of the pixel_size (default is 'nm')
        cmap: matplotlib colormap (default is 'gray')
        save: boolean variable if you want to save the image at destination (default is True)
        scale: scale of the image in percentile (default is (0,100))
    OUTPUT:
        no output, plots the image
    KG 01.2020
    """
    mi, ma = np.percentile(data[rang[0] : rang[1], :], scale)

    # plot the image and save it as .png
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(data, cmap=cmap, vmin=mi, vmax=ma, origin="lower")
    scalebar = ScaleBar(
        pixel_size,
        units=units,
        frameon=False,
        color=color,
        fixed_value=length_fraction,
        location=location,
    )
    ax.add_artist(scalebar)
    plt.tight_layout()
    if save:
        plt.savefig(destination, dpi=150)
    return fig, ax

# Load Data

Load the data.

1. Specify some parameters.
2. Load the header.
3. Load the image data.

# Functions

In [ ]:
# List of h5 keys for simple loading
mnemonics = dict()
mnemonics["image"] = "/entry1/APD/data"
mnemonics["energy"] = "/entry1/APD/energy"
mnemonics["sample_x"] = "/entry1/APD/sample_x"
mnemonics["sample_y"] = "/entry1/APD/sample_y"
mnemonics["magnetic_field"] = "/entry1/APD/magnetic_field"

In [ ]:
def load_image(im_id):
    fname = join(data_folder, "%s_%03d.hdf5" % (file_prefix, im_id))
    print("Loading: %s" % fname)

    if os.path.exists(fname):
        exist_flag = True
        with h5py.File(fname, "r") as f:
            image = f[mnemonics["image"]][()]
    else:
        print("File does not exist!")
        exist_flag = False
        image = np.array(np.zeros((2, 2)))

    return image, exist_flag


def load_images(im_id_list):
    images = []

    for im_id in im_id_list:
        image = load_image(im_id)
        images.append(image)

    return images


def get_pixel_size(im_id):
    fname = join(data_folder, "%s_%03d.hdf5" % (file_prefix, im_id))
    print("Loading: %s" % fname)
    with h5py.File(fname, "r") as f:
        sample_x = f[mnemonics["sample_x"]][()]
        sample_y = f[mnemonics["sample_y"]][()]

    # Calc pixel size from file
    pixel_size_x = np.round(np.mean(np.abs(sample_x[0:-2] - sample_x[1:-1])), 3)
    pixel_size_y = np.round(np.mean(np.abs(sample_y[0:-2] - sample_y[1:-1])), 3)

    return (pixel_size_y, pixel_size_x)

## Parameters

Specify the date from the data prefix. If you want to load an XMCD data set, set the boolean variable XMCD to True and specify positive and negative helicity image. Set the folder where the raw data is located (you might need to include a folder name according the the date via **'%i'%date**.

In [ ]:
# Just Basic folder structures for loading and saving
# For maxymus folder
date = "2024-04-16"

# Combine paths
file_prefix = "Sample_Image_%s" % date

# Input folder
data_folder = join("Z:\\data2", date)
# data_folder = os.getcwd()

# save folder
folder_save = r"C:\Users\maike\OneDrive\Desktop\Max-Born\Maxymus_April"
folder_save = (
    r"C:\Users\klose\MBI - Klose\Projects\BESSY_MAXYMUS_24_04_Skyrmion\Analysis\%s"
    % date
)
# folder_save = join(os.getcwd(), "analysis")

# Other
entryNumber = "entry1"
detector = "APD"

# Save Name
# Create save folder of it does not exit yet
if not (os.path.exists(folder_save)):
    print("Creating folder " + folder_save)
    os.mkdir(folder_save)

In [ ]:
# Define Scan id for loading
n_pos = 62

# do you have an xmcd data set? (record with both helicity)
XMCD = False

if XMCD is True:
    n_neg = 0

## Data

Load the data.

In [ ]:
if XMCD:
    pos = load_image(n_pos)
    neg = load_image(n_neg)
    data = np.log(pos / neg)
    topo = pos + neg

    figure_save = join(folder_save, "MAXYMUS_%s-%03d_%03d.png" % (date, n_pos, n_neg))
    print("Image saved as: %s" % figure_save)

elif XMCD is False:
    data = load_image(n_pos)
    figure_save = join(folder_save, "MAXYMUS_%s-%03d.png" % (date, n_pos))
    print("Image saved as: %s" % figure_save)

pixel_size, pixel_size_x = get_pixel_size(n_pos)
size = np.array(data.shape) / 50

# Plot single image

In [ ]:
# Plotting with cimshow
fig, ax = cimshow(np.flipud(data), cmap="gray")
ax.set_title("ImID: %03d" % n_pos)
scalebar = ScaleBar(
    pixel_size * 1e-3, units="um", frameon=False, color="r", fixed_value=1, location=1
)

plt.gca().add_artist(scalebar)
# fig.colorbar(mm, ax=ax)

In [ ]:
# Execute save
fname = join(folder_save, "MAXYMUS_%s-%03d.png" % (date, n_pos))
print("Saving: %s" % fname)
plt.savefig(fname, dpi=300)

In [ ]:
## Maxymus plotting
fname = join(folder_save, "MAXYMUS_%s-%03d.png" % (date, n_pos))
fig, ax = max_plot(
    data,
    fname,
    pixel_size * 1e-3,
    1,
    "r",
    size=(size[1], size[0]),
    units="um",
    save=False,
    scale=(1, 95),
    cmap="gray",
)
ax.set_title("ImID: %03d" % n_pos)
print("Saving: %s" % fname)
plt.tight_layout()
plt.savefig(fname, dpi=300)

# Loop this 

In [ ]:
#### Define image ids
im_ids = np.arange(198, 207)

# Loop over different ids
images = []
for im_id in im_ids:
    if XMCD:
        pos = load_image(im_id)
        neg = load_image(n_neg)
        data = np.log(pos / neg)
        topo = pos + neg

        figure_save = join(
            folder_save, "MAXYMUS_%s-%03d_%03d_looped.png" % (date, im_id, n_neg)
        )

    elif XMCD is False:
        data, exist_flag = load_image(im_id)
        figure_save = join(folder_save, "MAXYMUS_%s-%03d_looped.png" % (date, im_id))

    if exist_flag is True:
        images.append(data)
        size = np.array(data.shape) / 50

        # Plot and save
        fig, ax = max_plot(
            data,
            fname,
            pixel_size * 1e-3,
            1,
            "r",
            size=(size[1], size[0]),
            units="um",
            save=False,
            scale=(1, 99),  # (20, 61), #(6, 63)
            cmap="gray",
            # rang=(20, 40),
        )
        ax.set_title("ImID: %03d" % im_id)
        print("Saving: %s" % figure_save)
        plt.tight_layout()
        plt.savefig(figure_save, dpi=300)

In [ ]:
# Slide Show viewer of all images
fig, ax = cimshow(images, cmap="gray")
ax.set_title("All looped images")

# Pandas data handling playground

In [ ]:
def load_data_dict(file_prefix, scan_id):
    fname = join(data_folder, "%s_%03d.hdf5" % (file_prefix, scan_id))
    print("Loading: %s" % fname)

    data = dict()

    if os.path.exists(fname):
        with h5py.File(fname, "r") as f:
            data["scan_id"] = scan_id
            for key in f[entryNumber][detector].keys():
                data[key] = f[entryNumber][detector][key][()]
    else:
        print("File does not exist!")
        data["scan_id"] = scan_id

    return data


def load_data_to_pandas(scan_id_list, stack=False):
    if stack is True:
        prefix = "Sample_Stack_%s" % date
    elif stack is False:
        prefix = "Sample_Image_%s" % date

    data = []
    for scan_id in scan_id_list:
        data.append(load_data_dict(prefix, scan_id))

    df = pd.DataFrame(data)
    df.set_index("scan_id", inplace=True)

    return df


def get_title(scan_id, value, key):
    if key == "magnetic_field":
        title = "ScanId: %d, Magnetic Field: %.1f mT" % (scan_id, value)
    else:
        title = "ScanId: %d, %s: %.4f " % (scan_id, key, value)

    return title

In [ ]:
# Define for loading
scan_ids = [162, 194]  # np.arange(146, 190)
stack = True
scan_key = "magnetic_field"

# Loading of data
data = load_data_to_pandas(scan_ids, stack=stack)
data = data.dropna()
data

In [ ]:
# Slide show image viewer
fig, ax = cimshow(data["data"].values[0], cmap="gray")

In [ ]:
for j, scan_id in enumerate(scan_ids):
    # Plot hysteresis or any other key
    folder = join(folder_save, "%s_%d" % (scan_key, scan_id))
    if not (os.path.exists(folder)):
        print("Creating folder " + folder)
        os.makedirs(folder)

    flist = []
    for i, xdata in enumerate(data[scan_key].values[j]):
        image = data["data"].values[j][i]
        vmin, vmax = np.percentile(image, [1, 99])

        fig, ax = plt.subplots(figsize=(8, 8))
        ax.imshow(image, vmin=vmin, vmax=vmax, cmap="gray")
        ax.set_title(get_title(scan_id, xdata, scan_key))

        pixel_size = np.round(
            np.mean(
                np.abs(
                    data["sample_x"].values[j][0:-2] - data["sample_x"].values[j][1:-1]
                )
            ),
            3,
        )
        scalebar = ScaleBar(
            pixel_size,
            units="um",
            frameon=False,
            color="r",
            fixed_value=1,
            location=1,
        )

        plt.gca().add_artist(scalebar)

        fname = join(folder, "%s_ScanID_%03d_%03d.png" % (scan_key, scan_id, i))
        flist.append(fname)
        # print("Saving: %s" % fname)
        plt.savefig(fname)

    # Load files
    var = [imageio.imread(file) for file in flist]
    images = np.stack(var, axis=0)
    gif_path = join(folder_save, "%s_ScanID_%03d.gif" % (scan_key, scan_id))
    print(f"Now saving:", gif_path)
    imageio.mimsave(gif_path, var, fps=2)

In [ ]:
plt.close("all")

In [ ]:
# Plot x-y positions
xdata = data["sample_x"].tolist()
ydata = data["sample_y"].tolist()
print(
    "x-data: %0.2f, %0.2f , y-data: %0.2f, %0.2f"
    % (np.min(xdata), np.max(xdata), np.min(ydata), np.max(ydata))
)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle("ScanId: %s" % n_pos)
for idx in range(len(xdata)):
    ax[0].plot(xdata[idx], "o")
    ax[0].set_xlabel("Position idx")
    ax[0].set_ylabel("sample_x")
    ax[0].grid()

for idx in range(len(ydata)):
    ax[1].plot(ydata[idx], "o")
    ax[1].set_xlabel("Position idx")
    ax[1].set_ylabel("sample_y")
    ax[1].grid()